# Method to help identify candidate features for RTS Consideration

[Amazon Forecast](https://aws.amazon.com/forecast/) provides the ability to define and upload a special dataset type called [Related Time Series (RTS)](https://docs.aws.amazon.com/forecast/latest/dg/related-time-series-datasets.html) to help customers improve time-series model outcomes.  RTS is a set of features that move in time, similar to the time movement of the Target Time Series (TTS) dataset.  You may think of TTS as the dependent variable of the model and RTS as the independent variable(s).  The goal of RTS is to help inform the model by explaining some of the variability in the dependent variable and produce a model with better accuracy.

When customers use RTS, Amazon Forecast has a feature called [Predictor Explainability](https://docs.aws.amazon.com/forecast/latest/dg/predictor-explainability.html) which provides both visual and tabular feedback on which data features in the RTS were helpful to inform the model as shown in Figure 1.<br><br>


**Figure 1 - Built-in Predictor Explainability in Amazon Forecast**

![Predictor Explainability](./images/predictor-explainability.png) 
<br><br>

## Motivation for an exploratory candidate feature selection method

Sometimes customers have dozens or hundreds of candidate features of interest and ask for a quick way to help reduce the set by finding features that are not significant.  At the same time, a mutually exclusive secondary ask exists.  Customers want a method to identify independent RTS features that exhibit collinearity.  When candidate features in the RTS have a strong relationship, it can put the overall model at risk of overfitting.  The goal here is to remove all but one of the strongly correlated values.

There are many ways to accomplish these tasks of candidate selection.  The goal is to find features that help explain the TTS target value, but not explain other RTS variables.  This notebook provides rule-of-thumb guidance to assist with candidate selection.  You may choose to use other methods in addition to this method.  After having pared down the candidate set, you can create a new RTS set and import it into Amazon Forecast, where the [AutoPredictor](https://aws.amazon.com/blogs/machine-learning/new-amazon-forecast-api-that-creates-up-to-40-more-accurate-forecasts-and-provides-explainability/) will give you more precise feedback.  You can use the AutoPredictor metrics at the global or times-series level to understand if your proposed change was helpful and to what degree.

Finally, to be explicit this notebook and candidate selection process has no temporal nature implied.  This example simply compares the dependent and independent variables, **without time**, without sequence, and without any implied leads or lags.

## Setup

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
pd.options.display.float_format = '{:.3f}'.format

## Import RTS Data

This section will load RTS data from an open-source data set and generate features that should be flagged by the candidate selection proposed.

In [2]:
rts_column_list = ["location_id","item_id","checkout_price","base_price",
               "emailer_for_promotion","homepage_featured","timestamp"]

rts_dtype_dic= { "location_id":str,"item_id":str,"checkout_price":float,"base_price":float,
               "emailer_for_promotion":float,"homepage_featured":float,"timestamp":str}

rts_index = ['timestamp','location_id','item_id']

rts = pd.read_csv('./data/food-forecast-rts-uc1.csv',
                  index_col=rts_index,
                  skiprows=1,
                  names=rts_column_list,
                  dtype = rts_dtype_dic
                 )
print(rts.shape)            

(456547, 4)


Next, features will be created to ensure they are flagged for low importance and collinearity:
- feature X1 is created to simulate noise, via a random function
- feature X2 is created to be highly correlated to base price
- feature X3 is an engineered feature to inform discounted sales price

In [3]:
# seed the pseudorandom number generator
from random import seed
from random import random
# seed random number generator
seed(42)
rts['x1'] = [np.random.normal() for k in rts.index]
rts['x2'] = rts['base_price']*0.9
rts['x3'] = rts['checkout_price']/rts['base_price']

Preview the RTS 

In [4]:
rts.head(5)

,,,checkout_price,base_price,emailer_for_promotion,homepage_featured,x1,x2,x3
timestamp,location_id,item_id,,,,,,,
2018-10-16,10,1062,157.140,157.140,0.000,0.000,0.053,141.426,1.000
2018-10-30,10,1062,158.170,156.170,0.000,0.000,-1.791,140.553,1.013
2017-12-05,10,1062,159.080,182.390,0.000,0.000,0.387,164.151,0.872
2017-03-21,10,1062,159.080,183.330,0.000,0.000,-0.658,164.997,0.868
2017-06-20,10,1062,159.080,183.360,0.000,0.000,-0.217,165.024,0.868


## Import TTS Data

In [5]:
tts_column_list = ["location_id","item_id","target_value","timestamp"]
tts_dtype_dic= { "location_id":str,"item_id":str,"target_value":float,"timestamp":str}
tts_index = ['timestamp','location_id','item_id']

tts = pd.read_csv('./data/food-forecast-tts-uc1.csv',
                  index_col=tts_index,
                  skiprows=1,
                  names=tts_column_list,
                  dtype = tts_dtype_dic
                 )

print(tts.shape)

(456547, 1)


Preview the TTS 

In [6]:
tts.head(5)

target_value
timestamp  location_id item_id              
2016-11-08 55          1993          270.000
                       2539          189.000
                       2139           54.000
                       2631           40.000
                       1248           28.000

## Merge the TTS and RTS

Merge the two sets such that the Y and X values are in the same pandas row.  To keep things simple, the composite key of item and location are blended into a single feature.  You may choose to approach this in other ways.

In [7]:
merge_df = rts.join(tts)
merge_df = merge_df.reset_index()

# create composite key for simplicity
merge_df['location_id']=merge_df['location_id'].astype(str)
merge_df['item_id']=merge_df['item_id'].astype(str)
merge_df["key"] = merge_df[["location_id", "item_id"]].apply("-".join, axis=1)

# delete original singleton columns, obsolete with composite
merge_df.drop(columns=['location_id','item_id'] , inplace=True)

In [ ]:
# not needed, but if you want to ensure no time-series relatioships shuffle the data
#merge_df = merge_df.sample(frac=1, replace=True, random_state=1)

# Compute single regression per series

This step computes a multivariate linear regression per each series.  In this example, there are more than 3000 unique combinations of item and location originally.  This step computes a regression for each.  The time to complete will vary based on the size of your data, compute, and memory.  As delivered, alpha 0.05 is used to test for significance.

In [8]:
setlist={}
i=0

for ts in merge_df.key.unique():
    
    #create single ts dataframe
    final_df = merge_df[merge_df['key']==ts]
    
    X=final_df
    X=X.reset_index()
    
    y=X['target_value']
    y=pd.DataFrame(y)
    
    X.drop(columns=['target_value','index','key','timestamp'], inplace=True)

    # produce regression for single time series in loop
    model = sm.OLS(y,X)
    results = model.fit()
    results.params
    
    size=len(results.pvalues)

    # for each X variable in model, get statistics
    for v in range(0,size):
   
        # tally for statistically signifiant using alpha 0.05
        if results.pvalues[v]<=0.05:
            s=1
        else:
            s=0
                        
        list = {
        'key': ts,
        'variable': X.columns.values.tolist()[v],
        'coefficient': results.params[v],
        'p-value': results.pvalues[v],
        'significant': s,
        'count': 1
        }
        i=i+1
        setlist[i] = list 

result = pd.DataFrame()
result = result.from_dict(setlist, "index")

## How often do features significantly relate to the dependent variable?

In the next cells, statistical significance by feature is computed by regression.  The example highlights features that were significant in less than 20% of the time-series.  You can change the over-under, but potentially these candidate RTS values can be eliminated as you propose the best set of values for RTS inclusion.  Every use case is different; take care to change your over-under threshold.

In [9]:
print('Unique series in the dataset:',result.key.nunique())

regression_stats = pd.DataFrame(result.groupby('variable')['significant'].apply(lambda x : x.astype(int).sum()).sort_values())
regression_stats[regression_stats['significant']<=merge_df.key.nunique()*.2].sort_values(by="significant")

Unique series in the dataset: 3597


,significant
variable,
x1,181


In the above example, x1 was only significant in 192 of 3597 regressions.  Remember, x1 was built from a random number, so it is expected for this feature to be called out here.<br><br>On the other hand; below, these fields appear significant in at least 20% or more series. Change the threshold as appropriate for your use case.

In [10]:
regression_stats[regression_stats['significant']>merge_df.key.nunique()*.2].sort_values(by="significant")

,significant
variable,
emailer_for_promotion,2052
homepage_featured,2125
x3,2353
checkout_price,2888
base_price,3115
x2,3115


## Develop a Pearson correlation score for all bivariate pairs
The goal here is to identify high correlations between candidate X-values and remove them to prevent overfitting.

In [11]:
setlist={}
i=0

for ts in merge_df.key.unique():
    final_df = merge_df[merge_df['key']==ts]

    correlation_mat = final_df.corr()
    strong_pairs = correlation_mat[(abs(correlation_mat) > 0.7) & (abs(correlation_mat)<1)]
    strong_pairs = strong_pairs.reset_index().melt(id_vars='index')
    strong_pairs = strong_pairs.dropna().reset_index()

    for v in range(1,strong_pairs.shape[0]):
        
        if not (strong_pairs['index'][v] !='target_value') or (strong_pairs['index'][v] != 'target_value'):
            list = {
            'key': ts,
            'index': strong_pairs['index'][v],
            'variable': strong_pairs['variable'][v],
            'coefficient': strong_pairs['value'][v],
            'count': 1
            }
            i=i+1
            setlist[i] = list 
     
result_corr = pd.DataFrame()
result_corr = result_corr.from_dict(setlist, "index")

## Inspect the Correlation Results

As delivered, if more than 20% of the series are highly correlated, they are highlighted below.  The idea is to apply business logic and domain knowledge to determine if one or more of these should be removed to prevent overfitting of the model.

In this example, x2 was created as a function of price, which explains the high Pearson finding.
Variable x3 was created from other features too.  Feature x3 represents a discounted price.  It may be plausible to use this feature or the two features used to manufacture it -- but not both.

In [12]:
stats = pd.DataFrame(result_corr.groupby(['index', 'variable'])["coefficient"].apply(lambda x : x.astype(int).count())).reset_index()
stats[stats['coefficient']>=merge_df.key.nunique()*.2]

,index,variable,coefficient
3,base_price,x2,1647
11,checkout_price,x3,2386
15,emailer_for_promotion,x3,811
34,x2,base_price,1321
42,x3,emailer_for_promotion,808
44,x3,target_value,720
